In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install einops
%pip install wandb

In [ ]:
%cd drive/MyDrive/Colab Notebooks/ViT

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/ViT'
/content/drive/MyDrive/Colab Notebooks/ViT


In [ ]:
from model.vit import ViT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [ ]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [ ]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 64,
    "patch_size" : 16,
    "num_classes" : 100,
    "head_dim" : 64,
    "dim" : 512,
    "depth" : 6,
    "heads" : 16,
    "mlp_dim" : 2048,
    "channels" : 3,
    "dropout" : 0.1,
    "emb_dropout": 0.1,
    }


# Training settings
import wandb
wandb.init(config=config, project="Cifar100_ViT", entity='NoguNogu',name='Vanila')

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms


train_transforms = transforms.Compose([transforms.Resize((64, 64)),
                                       transforms.RandomResizedCrop(64),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()
                                       ])

val_transforms = transforms.Compose([
                                     transforms.Resize((64, 64)),
                                     transforms.RandomResizedCrop(64),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor()
                                     ])


trainset = datasets.CIFAR100(
    root='./data/Cifar100',
    train=True,
    download=False,
    transform=train_transforms)
    
testset = datasets.CIFAR100(
    root='./data/Cifar100',
    train=False,
    download=False,
    transform=val_transforms)

trainloader = DataLoader(trainset,
                         batch_size=64,
                         shuffle=True,
                         num_workers=2)

testloader = DataLoader(testset,
                         batch_size=64,
                         shuffle=False,
                         num_workers=2)


In [ ]:
model = ViT(
    dim = config['dim'],
    image_size = config['image_size'],
    patch_size = config['patch_size'],
    num_classes= config['num_classes'],
    channels= config['channels'],
    depth= config['depth'],
    heads= config['heads'],
    mlp_dim = config['mlp_dim']
    ).to(device)

In [ ]:
from torchsummary import summary
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1              [-1, 16, 768]               0
            Linear-2              [-1, 16, 512]         393,728
           Dropout-3              [-1, 17, 512]               0
         LayerNorm-4              [-1, 17, 512]           1,024
            Linear-5             [-1, 17, 3072]       1,572,864
           Softmax-6           [-1, 16, 17, 17]               0
            Linear-7              [-1, 17, 512]         524,800
           Dropout-8              [-1, 17, 512]               0
         Attention-9              [-1, 17, 512]               0
          PreNorm-10              [-1, 17, 512]               0
        LayerNorm-11              [-1, 17, 512]           1,024
           Linear-12             [-1, 17, 2048]       1,050,624
             GELU-13             [-1, 17, 2048]               0
          Dropout-14             [-1, 1

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [ ]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 4.2866 - acc: 0.0479 - val_loss : 4.0723 - val_acc: 0.0810




Epoch : 2 - loss : 3.9330 - acc: 0.1024 - val_loss : 3.8224 - val_acc: 0.1232




Epoch : 3 - loss : 3.7169 - acc: 0.1346 - val_loss : 3.6393 - val_acc: 0.1452




Epoch : 4 - loss : 3.5631 - acc: 0.1610 - val_loss : 3.5119 - val_acc: 0.1689




Epoch : 5 - loss : 3.4460 - acc: 0.1824 - val_loss : 3.4126 - val_acc: 0.1948



...


Epoch : 16 - loss : 2.8564 - acc: 0.2932 - val_loss : 2.9958 - val_acc: 0.2696




Epoch : 17 - loss : 2.8143 - acc: 0.3039 - val_loss : 2.9821 - val_acc: 0.2764




Epoch : 18 - loss : 2.7916 - acc: 0.3086 - val_loss : 2.9454 - val_acc: 0.2848




Epoch : 19 - loss : 2.7608 - acc: 0.3146 - val_loss : 2.9196 - val_acc: 0.2899




Epoch : 20 - loss : 2.7289 - acc: 0.3197 - val_loss : 2.9031 - val_acc: 0.2959



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat May 15 06:56:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    54W / 300W |   2413MiB / 16160MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------